In [ ]:
import mlflow
from dotenv import load_dotenv
import openai

load_dotenv()

MODEL='gpt-oss-20b-local'
PROMPTS = "prompts:/qa-agent-user-prompt@latest"
TEMPERATURE = 0

# Step 1: Register your initial prompt
# 建立 user prompt，如果已存在則不重複建立
def create_or_load_prompt():
    user_prompt = mlflow.genai.load_prompt(PROMPTS)
    if not user_prompt:
        mlflow.genai.register_prompt(
            name="qa-agent-user-prompt",
            template="""You are a helpful assistant. Answer questions concisely in Traditional Chinese.
Question: {{question}}
Answer:""",
        )
        print("Prompt not found. Registered a new prompt.")
    return mlflow.genai.load_prompt(PROMPTS)

# Step 2: Create a prediction function
def predict_fn(question: str, model: str) -> str:
    # Load prompt from registry
    user_prompt = create_or_load_prompt()
    formatted_prompt = user_prompt.format(question=question)

    # 檢查 formatted_prompt 是列表還是字串
    if isinstance(formatted_prompt, list):
        messages = formatted_prompt
    else:
        messages = [{"role": "user", "content": formatted_prompt}]

    response = openai.OpenAI().chat.completions.create(
        model=model, messages=messages, temperature=TEMPERATURE,
    )
    return response.choices[0].message.content

# Step 3: Define scorers
print(create_or_load_prompt().template)

result = predict_fn("?", model=MODEL)
print(result)

In [ ]:
import requests
import os

# 從 localhost:4000 取得可用模型列表
api_key = os.getenv("LITELLM_API_KEY")
response = requests.get("http://localhost:4000/v1/models", headers={"Authorization": f"Bearer {api_key}"})
available_models = response.json().get("data", [])


In [ ]:
print("Available models from localhost:4000:")
for model_info in available_models[0:8]:
    print(f"- {model_info['id']}")

In [ ]:
model_list = ["gpt-oss-20b-local", ]
model_list += [model_info['id'] for model_info in available_models[0:8]]
for model in model_list:
    try:
        result = predict_fn("你的知識截止日是什麼時候？", model=model)
        print(f"Model: {model}, Answer: {result}")
    except Exception as e:
        print(f"Error with model {model}: {e}")

In [ ]:
model_list = []
model_list += [model_info['id'] for model_info in available_models[9:]]
for model in model_list:
    try:
        result = predict_fn("你的知識截止日是什麼時候？", model=model)
        print(f"Model: {model}, Answer: {result}")
    except Exception as e:
        print(f"Error with model {model}: {e}")